In [1]:
import copy
import gc
import json

import tempfile
import os

# 检查 tempfile 模块使用的临时文件目录
temp_dir = tempfile.gettempdir()
print("Updated temp directory:", temp_dir)

import sys
import importlib

new_path = '/data/mr423/project/ageClassification/'
if new_path not in sys.path:
    sys.path.insert(0, new_path)

# relaod the scgpt files
import scgpt
print("scgpt location: ", scgpt.__file__)
importlib.reload(scgpt)

Updated temp directory: /data/mr423/tmp
scgpt location:  /data/mr423/project/code/ageClassification/scgpt/__init__.py


<module 'scgpt' from '/data/mr423/project/code/ageClassification/scgpt/__init__.py'>

In [2]:
from pathlib import Path
import shutil
import sys
import time
import traceback
from typing import List, Tuple, Dict, Union, Optional
import warnings
import pandas as pd
# from . import asyn
import pickle
import torch
from anndata import AnnData
import scanpy as sc
import scvi
import seaborn as sns
import numpy as np
import wandb
from scipy.sparse import issparse
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from torchtext.vocab import Vocab
from torchtext._torchtext import (
    Vocab as VocabPybind,
)
from sklearn.metrics import confusion_matrix

sys.path.insert(0, "../")
import scgpt as scg

import argparse
from scgpt.model import TransformerModel, AdversarialDiscriminator
from scgpt.tokenizer import tokenize_and_pad_batch, random_mask_value
from scgpt.loss import (
    masked_mse_loss,
    masked_relative_error,
    criterion_neg_log_bernoulli,
)
from scgpt.tokenizer.gene_tokenizer import GeneVocab
from scgpt.preprocess import Preprocessor
from scgpt import SubsetsBatchSampler
from scgpt.utils import set_seed, category_str2int, eval_scib_metrics

sc.set_figure_params(figsize=(6, 6))
os.environ["KMP_WARNINGS"] = "off"
warnings.filterwarnings('ignore')

os.environ["WANDB_MODE"]= "offline"

Global seed set to 0


In [4]:
######################################################################
# Settings for wandb mentior
######################################################################

hyperparameter_defaults = dict(
    seed=0,
    do_train=True,
    load_model="/data/mr423/project/pre_trained_model/scGPT_human",
    n_bins=101,

    epochs=1, # 50 !!!!!!!!!!!!  test only
    lr=0.001,
    batch_size=32,   # 128 !!!!!!!!!!!!  test only

    layer_size=128, # 128
    nlayers=4,  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
    nhead=8,  # number of heads in nn.MultiheadAttention
    
    dropout=0.0,  # dropout probability

    use_fast_transformer=True,
    pre_norm=False,
    amp=True,  # Automatic Mixed Precision
    freeze = True, #freeze
)

run = wandb.init(
    config=hyperparameter_defaults,
    project="age_clusters-test",
    reinit=True,
    settings=wandb.Settings(start_method="fork"),
)
config = wandb.config
print(config)

set_seed(config.seed)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


{'seed': 0, 'do_train': True, 'load_model': '/data/mr423/project/pre_trained_model/scGPT_human', 'n_bins': 101, 'epochs': 1, 'lr': 0.001, 'batch_size': 32, 'layer_size': 128, 'nlayers': 4, 'nhead': 8, 'dropout': 0.0, 'use_fast_transformer': True, 'pre_norm': False, 'amp': True, 'freeze': True}


In [6]:
######################################################################
# Settings for input and preprocessing
######################################################################
pad_token = "<pad>"
special_tokens = [pad_token, "<cls>", "<eoc>"]
mask_value = "auto"  # for masked values, now it should always be auto

max_seq_len = 3001
n_bins = config.n_bins

# input/output representation
input_style = "binned"  # "normed_raw", "log1p", or "binned"
input_emb_style = "category"  # "category" or "continuous" or "scaling"
cell_emb_style = "cls"  # "avg-pool" or "w-pool" or "cls"


# settings for training
CLS = True  # celltype classification objective

In [7]:
######################################################################
# Settings for optimizer
######################################################################
lr = config.lr  # TODO: test learning rate ratio between two tasks
batch_size = config.batch_size
eval_batch_size = config.batch_size
epochs = config.epochs
early_stop = 5

In [10]:
######################################################################
# Settings for the model
######################################################################
use_fast_transformer = config.use_fast_transformer
fast_transformer_backend = "flash"  # "linear" or "flash"


embsize = config.layer_size  # embedding dimension
d_hid = config.layer_size  # dimension of the feedforward network in TransformerEncoder
nlayers = config.nlayers  # number of TransformerEncoderLayer in TransformerEncoder
nhead = config.nhead  # number of heads in nn.MultiheadAttention
dropout = config.dropout  # dropout probability

In [11]:
######################################################################
# Validate the settings
######################################################################
# %% validate settings
assert input_style in ["normed_raw", "log1p", "binned"]
# assert output_style in ["normed_raw", "log1p", "binned"]

assert input_emb_style in ["category", "continuous", "scaling"]
if input_style == "binned":
    if input_emb_style == "scaling":
        raise ValueError("input_emb_style `scaling` is not supported for binned input.")
elif input_style == "log1p" or input_style == "normed_raw":
    if input_emb_style == "category":
        raise ValueError(
            "input_emb_style `category` is not supported for log1p or normed_raw input."
        )

if input_emb_style == "category":
    mask_value = n_bins + 1
    pad_value = n_bins  # for padding gene expr values
    n_input_bins = n_bins + 2
else:
    mask_value = -1
    pad_value = -2
    n_input_bins = n_bins

In [12]:
######################################################################
# Settings for the running recording
######################################################################
dataset_name = 'biobank-age-cluster'
save_dir = Path(f"../save/dev_{dataset_name}-{time.strftime('%b%d-%H-%M')}/")
save_dir.mkdir(parents=True, exist_ok=True)

print(f"save to {save_dir}")
logger = scg.logger
scg.utils.add_file_handler(logger, save_dir / "run.log")

save to ../save/dev_biobank-age-cluster-Aug23-23-06


In [14]:
######################################################################
# Data loading
######################################################################
adata = sc.read("/data/mr423/project/data/3-OLINK_data_train_withOutlier_all.h5ad")
adata_test = sc.read("/data/mr423/project/data/3-OLINK_data_test_withOutlier_all.h5ad")

print(adata.shape)
print(adata_test.shape)

adata.obs["batch_id"]  = adata.obs["str_batch"] = "0"
adata_test.obs["batch_id"]  = adata_test.obs["str_batch"] = "1" 

adata.var.set_index(adata.var["gene_name"], inplace=True)
adata_test.var.set_index(adata.var["gene_name"], inplace=True)

data_is_raw = False
filter_gene_by_counts = False
adata_test_raw = adata_test.copy()
adata = adata.concatenate(adata_test, batch_key="str_batch")

# make the batch category column
batch_id_labels = adata.obs["str_batch"].astype("category").cat.codes.values
adata.obs["batch_id"] = batch_id_labels


ageGroup_id_labels = adata.obs["Age_Group"].astype("category").cat.codes.values
# ageGroup_types = adata.obs["Age_Group"].unique()

num_types = len(np.unique(ageGroup_id_labels))
# id2type = dict(enumerate(adata.obs["Age_Group"].astype("category").cat.categories))
# print(id2type)

adata.obs["ageGroup_id"] = ageGroup_id_labels

adata.var["gene_name"] = adata.var.index.tolist()

(37304, 2919)
(4145, 2919)


In [15]:
adata.obs

,sex,DoB_Year,DoB_Month,DoB_Day,DoB,Date_Attend,age,Age_Group,batch_id,str_batch,ageGroup_id
Id,,,,,,,,,,,
2144829-0,0,1939,1,15,1939-01-15,2007-11-16,68.835044,60-70,0,0,3
3154285-0,0,1945,1,15,1945-01-15,2007-07-20,62.507871,60-70,0,0,3
1679423-0,1,1945,11,15,1945-11-15,2009-05-19,63.507187,60-70,0,0,3
1172610-0,1,1941,12,15,1941-12-15,2009-09-23,67.772758,60-70,0,0,3
4011532-0,1,1954,1,15,1954-01-15,2009-10-20,55.761807,50-60,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
2503594-1,0,1947,2,15,1947-02-15,2009-08-13,62.491444,60-70,1,1,3
3494250-1,1,1945,12,15,1945-12-15,2009-11-10,63.904175,60-70,1,1,3
5746191-1,1,1951,12,15,1951-12-15,2009-06-26,57.530459,50-60,1,1,2
